In [1]:
import matplotlib.pyplot as plt
import numpy as np
import skimage.io as io
import skimage.draw as draw
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import datetime
from Scanner import *
import pydicom


%matplotlib inline

Funkcje pomocnicze

In [2]:
def conductScan(nDet: int, phi: float, noViews: int, scanner: Scanner, useFilter: bool):
    s.set_no_detectors(nDet)
    s.set_phi(phi)
    s.set_no_views(noViews)
    if(useFilter):
        scanner.gen_filter(21)
    else:
        s.gen_filter(0)
    s.gen_spect()
    
    io.imshow(s.get_spect())
    plt.show()
    
    io.imshow(s.get_restored_img())
    plt.show()
    
    scanner.view_mse()
    plt.show()

    interact(show_steps, number_of_step=(0, s.get_no_steps() - 1))
    
def show_steps(number_of_step: int):
    io.imshow(s.get_step_img(number_of_step))
    plt.show()
    
    
def readImage(name: str, scanner: Scanner):
    img = io.imread(name, as_gray=True)
    scanner.load_image(img)
    io.imshow(img)
    plt.show()
    
def readDicom(path: str, show_patient_data: bool, show_date: bool, show_comments: bool):
    dcm = pydicom.dcmread(path)
    image = dcm.pixel_array
    
    io.imshow(image)
    
    if show_patient_data:
        print('''Dane pacjenta:\nImię i nazwisko: {}\nWiek: {}
              Płeć: {}\nID: {}\n'''.format(dcm.PatientName, dcm.PatientAge, dcm.PatientSex, dcm.PatientID))
    
    if show_date:
        print("Data badania: {}\nGodzina badania: {}\n".format(dcm.ContentDate, dcm.ContentTime))
        
    if show_comments:
        print("Komentarze: {}\n".format(dcm.ImageComments))

# Wczytanie pliku do przeskanowania

In [3]:
s = Scanner()
interact(readImage, name=["Kropka.jpg", "CT_ScoutView.jpg", "CT_ScoutView-large.jpg",
                          "Kolo.jpg", "Kwadraty2.jpg", "Paski2.jpg", "SADDLE_PE.JPG", "SADDLE_PE-large.JPG",
                         "Shepp_logan.jpg"],
         scanner=fixed(s))

interactive(children=(Dropdown(description='name', options=('Kropka.jpg', 'CT_ScoutView.jpg', 'CT_ScoutView-la…

<function __main__.readImage(name: str, scanner: Scanner.Scanner)>

# Przeprowadzenie skanu i wyświetlenie wyników

Po przeprowadzeniu skanu pojawi się przeglądarka kolejnych kroków tworzenia skanu

In [4]:
interact_manual(conductScan, nDet=(1, 360), phi=(np.pi/180, 2* np.pi, np.pi/180),  
         noViews=(5, 360, 5), scanner=fixed(s), useFilter=True)

interactive(children=(IntSlider(value=180, description='nDet', max=360, min=1), FloatSlider(value=3.1415926535…

<function __main__.conductScan(nDet: int, phi: float, noViews: int, scanner: Scanner.Scanner, useFilter: bool)>

# Zapisy wyniku skanu do pliku DICOM

In [5]:
interact_manual(s.save_to_dicom, filename="new.dcm", patient_name="DOE^John", patient_sex="male",
        patient_age="42", patient_id="123", scan_date=widgets.DatePicker(value=datetime.datetime.now()), 
         image_comments="")

interactive(children=(Text(value='new.dcm', description='filename'), Text(value='DOE^John', description='patie…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

# Wczytanie pliku DICOM i wyświetlenie obrazu

In [6]:
interact_manual(readDicom, path="new.dcm", show_patient_data=False, show_date=False, show_comments=False)

interactive(children=(Text(value='new.dcm', description='path'), Checkbox(value=False, description='show_patie…

<function __main__.readDicom(path: str, show_patient_data: bool, show_date: bool, show_comments: bool)>